In [1]:
import math
import shutil
from pathlib import Path
from uuid import uuid4

import pandas as pd
from slugify import slugify
# go to https://ford-parts.com.ru/bitrix/admin/iblock_data_export.php?lang=ru
# Каталог -> Каталог товаров и Торговые предложения
# Вытащить категории, сабкатегории и скрестить с новой версией в Postgres
#

In [27]:
catalogue = pd.read_csv("data/category.csv")

catalogue = catalogue[
    (catalogue["IE_NAME"] != "1")
    & (catalogue["IE_NAME"] != "0")
    & (catalogue["IE_NAME"] != "01")
    & (catalogue["IC_GROUP1"].notna())
    & (catalogue["IC_GROUP2"].notna())
]

In [28]:
catalogue = catalogue[
    [
        "IE_XML_ID",
        "IE_NAME",
        "IE_PREVIEW_PICTURE",
        "IE_PREVIEW_TEXT",
        "IC_GROUP1",
        "IC_GROUP2",
    ]
].rename(
    columns={
        "IE_XML_ID": "bitrix_id",
        "IE_NAME": "name",
        "IE_PREVIEW_PICTURE": "old_image",
        "IE_PREVIEW_TEXT": "cross_number",
        "IC_GROUP1": "category",
        "IC_GROUP2": "sub_category",
    }
)
ids = [str(uuid4()) for i in range(len(catalogue))]
catalogue["id"] = ids
catalogue["slug"] = catalogue["name"]
catalogue["slug"] = [slugify(n, lowercase=False) for n in catalogue["slug"]]
catalogue["slug"] = [s + "-" + str(uuid4())[:6] for s in catalogue["slug"].str.lower()]

In [29]:
catalogue[
    catalogue["slug"].isin(catalogue["slug"].value_counts()[:40].index)
].sort_values("name").to_csv("seo_duplicates.csv", index=False)

In [30]:
catalogue

,bitrix_id,name,old_image,cross_number,category,sub_category,id,slug
1012,F8-2,Focus II-III / C-Max / Kuga / Mondeo / S-Max 0...,/upload/iblock/965/h6v1v60gi1ryckdh0tc7ay6ltnz...,"1930055, 1743698, 8M51R442A66EA",Кузов,Механизмы замков,cf4ef27c-8fcf-4ad8-82ff-a0790f2783d5,focus ii-iii / c-max / kuga / mondeo / s-max 0...
1013,F8-3,Focus II-III / C-Max / Kuga / Mondeo / S-Max 0...,/upload/iblock/da3/klvvib5jemjl1rp5rfraxiri73a...,"1859161, 8M51R442A66AC, 8M51 R442A66 DB, 15704...",Кузов,Механизмы замков,6611f1ab-c720-4de3-ad0b-268997108ef2,focus ii-iii / c-max / kuga / mondeo / s-max 0...
1014,F8-6,Механизм замка Connect задней (высокой) двери ...,NaN,"1419513, 2T14 V43287 BP",Кузов,Механизмы замков,204385f2-978d-4a59-b52d-ca6d5b5ddb93,механизм замка connect задней (высокой) двери ...
1015,F8-7,Механизм замка L ЛЕВОЙ выдвижной длинной двер...,/upload/iblock/3bc/4b6qcru899yl23shyk2ndyfvic7...,"1766561, YC15-V264A33-BK",Кузов,Механизмы замков,6362a95b-31ec-489a-885b-275f8ce21f2d,механизм замка l левой выдвижной длинной двер...
1016,F8-8,Механизм замка R правой выдвижной длинной двер...,/upload/iblock/f18/n4qjnqyb01i5awupzy87xbd8w15...,YC15 B264A00 BL N - FSE 11-321-050 - 1C15 B264...,Кузов,Механизмы замков,6abe47b9-ae0a-47a1-bcb3-28f9d17d8df7,механизм замка r правой выдвижной длинной двер...
...,...,...,...,...,...,...,...,...
9650,F22-467,Датчик давления выхлопных газов Ford Transit 2...,NaN,"1600077, 6G9N-5L200-AB",Система топливная,Датчик давления воздуха (MAP / наддув / выхлоп),0b554a17-fc73-4428-a562-60bd06c5b351,датчик давления выхлопных газов ford transit 2...
9651,F22-474,Датчик давления наддува FORD 1.6 TDCI,NaN,"1256481, 1468879, 1231463, 1256481, MAZDA Y601...",Система топливная,Датчик давления воздуха (MAP / наддув / выхлоп),1056d5f9-6fea-435b-b724-91a561f80167,датчик давления наддува ford 1.6 tdci-942308
9652,F22-300,Датчик давления фреона Mondeo 1993-2000 / Tran...,/upload/iblock/dd3/dlji06uet8z2kkjp92a1r03qjhy...,"95BW19E561AA, 3317869, 2S6H19E561AA, 1217951, ...",Электрооборудование,Датчики кондиционера,87793abb-f7e7-4770-9bbe-1950f3f82edb,датчик давления фреона mondeo 1993-2000 / tran...
9653,F22-473,Датчик давления фреона,NaN,"95BW 19E561 AA , 1016565",Электрооборудование,Датчики кондиционера,004a3178-3557-4604-aade-8f32768291b0,датчик давления фреона -fb5cf8


In [31]:
# catalogue['name'].value_counts()

## Временное решение для дубликатов (пропустить потом)

In [32]:
# tmp_counts = catalogue["name"].value_counts().reset_index()

In [33]:
# tmp_counts = tmp_counts[(tmp_counts["count"] > 1) & (tmp_counts["name"] != "1")]

In [34]:
# duplicates_to_export = catalogue[catalogue["name"].isin(tmp_counts["name"])].copy()
#
# duplicates_to_export["link"] = (
#         "https://ford-parts.com.ru/catalog/"
#         + duplicates_to_export["bitrix_id"].astype(str)
#         + "/"
# )


In [35]:
# duplicates_to_export.to_excel("data/duplicates.xlsx", index=False)

## Создать категории и сабкатегории

In [36]:
category_images = pd.read_csv("data/images_categories_tcf.csv")

In [37]:
category = pd.DataFrame(catalogue["category"].drop_duplicates())
category = category.rename(columns={"category": "name"})
ids = [str(uuid4()) for i in range(len(category))]
category["category_id"] = ids
category = category.merge(category_images, on="name", how="left")
category

,name,category_id,slug,old_image
0,Кузов,0cc8acea-07e7-4a39-8ba1-9ca3a5344efc,kuzov-ford,/upload/iblock/eb0/a7whmp91qy522054431cyx4dcb1...
1,Двигатель,d2ff4e0f-d89d-43e9-b8b2-fda6025837f5,dvigatel-ford,/upload/iblock/62b/iuf1syct13rhi50he2p4x3ns068...
2,Аксессуары,07332d18-a484-4002-9b9c-b0f1609d018a,aksessuary-ford,/upload/iblock/4ac/eo5pnechgqjs39tiqk71bruzl5z...
3,Система выхлопная и Турбины,6489c75f-176d-4d0b-b523-4ea9362a3175,sistema-vyhlopnaya-i-turbiny-ford,/upload/iblock/65b/xkh1awpw1dlkna93chdivy7pe05...
4,Подвеска,1329501b-80b9-4145-ac4e-5fb08426f07f,podveska-ford,/upload/iblock/ef3/lke05y6nond3gcnsle9xd2q4swb...
5,Система охлаждения,fc490adb-738f-4188-9833-5c4bb7d3a295,sistema-ohlazhdeniya-ford,/upload/iblock/b36/3x283iz4lbl98hfjum2sr1fup97...
6,Рулевое управление,934a4913-1111-4b86-9d79-52dcf9196e2a,rulevoe-upravlenie-ford,/upload/iblock/af1/mlo1135rjsuqv24ijzv18801sns...
7,Система зажигания и управ-я,ce7957c6-648d-4aa9-8b8e-65bbe3ef027a,sistema-zazhiganiya-i-uprav-ya-ford,/upload/iblock/da3/tkw3vi1jlgxdxrwb0p35l1ipbdl...
8,Стартеры и их компоненты,39603224-8adc-4942-9b25-ba68efc06b07,startery-i-ih-komponenty-ford,/upload/iblock/deb/2jk27o1ymwwo1fc0tzllhi184ii...
9,Колеса,12afe774-e308-4b7f-bb4c-749f97f03a2d,kolesa-ford,/upload/iblock/619/7s44rkdn1v0h0njquwe90pjyma8...


In [38]:
sub_category = pd.DataFrame(catalogue["sub_category"].drop_duplicates())
sub_category = sub_category.rename(columns={"sub_category": "name"})
ids = [str(uuid4()) for i in range(len(sub_category))]
sub_category["sub_category_id"] = ids
sub_category["slug"] = [slugify(n) for n in sub_category["name"].str.lower()]
sub_category = sub_category.merge(
    category_images[["name", "old_image"]], on="name", how="left"
)

In [39]:
one_big_table = catalogue.merge(
    category, left_on="category", right_on="name", how="left"
).merge(sub_category, left_on="sub_category", right_on="name", how="left")

In [40]:
categories = one_big_table[["category_id", "name_y", "slug_y", "old_image_y"]].copy()
categories = categories.rename(
    columns={
        "category_id": "id",
        "name_y": "name",
        "slug_y": "slug",
        "old_image_y": "old_image",
    }
)
categories = categories.drop_duplicates("name")

In [41]:
old_dir = Path("./upload")
new_dir = Path("./category_img")
new_dir.mkdir(parents=True, exist_ok=True)


def copy_and_rename_image(row) -> str:
    old_value = row["old_image"]

    # пропуск пустых или невалидных значений
    if (
        not isinstance(old_value, str)
        or not old_value.strip()
        or (isinstance(old_value, float) and math.isnan(old_value))
    ):
        return None

    old_value = "." + old_value

    old_path = Path(old_value)
    print(old_path)
    if not old_path.exists():
        print(f"⚠️ Файл не найден: {old_path}")
        return None

    ext = old_path.suffix.lower()
    new_filename = f"{row['id']}{ext}"
    new_path = new_dir / new_filename

    shutil.copy2(old_path, new_path)
    return new_filename


# применяем к датафрейму
categories["image_url"] = categories.apply(copy_and_rename_image, axis=1)

upload\iblock\eb0\a7whmp91qy522054431cyx4dcb1ffcus.PNG
upload\iblock\62b\iuf1syct13rhi50he2p4x3ns0688u5rb.png
upload\iblock\4ac\eo5pnechgqjs39tiqk71bruzl5zyqifl.PNG
upload\iblock\65b\xkh1awpw1dlkna93chdivy7pe052l7y6.PNG
upload\iblock\ef3\lke05y6nond3gcnsle9xd2q4swb2xi3p.PNG
upload\iblock\b36\3x283iz4lbl98hfjum2sr1fup979ghnr.PNG
upload\iblock\af1\mlo1135rjsuqv24ijzv18801snshs6ca.PNG
upload\iblock\da3\tkw3vi1jlgxdxrwb0p35l1ipbdlfopm9.PNG
upload\iblock\deb\2jk27o1ymwwo1fc0tzllhi184iix2o0v.PNG
upload\iblock\619\7s44rkdn1v0h0njquwe90pjyma8pbd46.PNG
upload\iblock\68e\ug1toh7bqcxmvezcg82ozdy2s3zte2p8.PNG
upload\iblock\f79\vkmipce0ynbmq18cjmfmook8w8owz898.PNG
upload\iblock\e67\ib6btlm17dy7fesheum3lmnrsnj4z3q7.png
upload\iblock\72f\ihzz2seihlpibgr6dley1eq8tsu1c1qt.PNG
upload\iblock\f42\rl8be0o3mmipdcvn2owatz1l47opw2ec.PNG
upload\iblock\b9c\ziq173396tc23hvbzf020ax4fq6hu22w.PNG
upload\iblock\b6a\l5v2h0rsfnw11asi9jwmdn4ytdgykn3p\Топливная_система.jpg
upload\iblock\3bb\wi5n804o3qdhwgfp09ll3eeh75ldl

In [42]:
categories["image_url"] = (
    "https://storage.yandexcloud.net/tcf-images/categories/" + categories["image_url"]
)
categories["image_url"] = categories["image_url"].fillna(
    "https://storage.yandexcloud.net/tcf-images/default.svg"
)
categories.drop(columns=["old_image"]).to_csv("out/categories.csv", index=False)

In [43]:
sub_categories = one_big_table[
    ["sub_category_id", "category_id", "name", "slug", "old_image"]
].copy()
sub_categories = sub_categories.rename(
    columns={
        "sub_category_id": "id",
        "name": "name",
        "slug": "slug",
        "old_image": "old_image",
    }
)

In [44]:
old_dir = Path("./upload")
new_dir = Path("./sub_category_img")
new_dir.mkdir(parents=True, exist_ok=True)


def copy_and_rename_image(row) -> str:
    old_value = row["old_image"]

    # пропуск пустых или невалидных значений
    if (
        not isinstance(old_value, str)
        or not old_value.strip()
        or (isinstance(old_value, float) and math.isnan(old_value))
    ):
        return None

    old_value = "." + old_value

    old_path = Path(old_value)
    print(old_path)
    if not old_path.exists():
        print(f"⚠️ Файл не найден: {old_path}")
        return None

    ext = old_path.suffix.lower()
    new_filename = f"{row['id']}{ext}"
    new_path = new_dir / new_filename

    shutil.copy2(old_path, new_path)
    return new_filename


# применяем к датафрейму
sub_categories["image_url"] = sub_categories.apply(copy_and_rename_image, axis=1)

upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\ibl

In [45]:
sub_categories["image_url"] = (
    "https://storage.yandexcloud.net/tcf-images/sub_categories/"
    + sub_categories["image_url"]
)
sub_categories["image_url"] = sub_categories["image_url"].fillna(
    "https://storage.yandexcloud.net/tcf-images/default.svg"
)
sub_categories.drop(columns=["old_image"]).drop_duplicates("name").to_csv(
    "out/sub_categories.csv", index=False
)

In [46]:
## PRODUCTS

In [47]:
products = one_big_table[
    [
        "bitrix_id",
        "id",
        "sub_category_id",
        "name_x",
        "slug_x",
        "cross_number",
        "old_image_x",
    ]
].copy()
products = products.rename(
    columns={"name_x": "name", "slug_x": "slug", "old_image_x": "old_image"}
)

In [48]:
old_dir = Path("./upload")
new_dir = Path("./products_img")
new_dir.mkdir(parents=True, exist_ok=True)


def copy_and_rename_image(row) -> str:
    old_value = row["old_image"]

    # пропуск пустых или невалидных значений
    if (
        not isinstance(old_value, str)
        or not old_value.strip()
        or (isinstance(old_value, float) and math.isnan(old_value))
    ):
        return None

    old_value = "." + old_value

    old_path = Path(old_value)
    print(old_path)
    if not old_path.exists():
        print(f"⚠️ Файл не найден: {old_path}")
        return None

    ext = old_path.suffix.lower()
    new_filename = f"{row['id']}{ext}"
    new_path = new_dir / new_filename

    shutil.copy2(old_path, new_path)
    return new_filename


# применяем к датафрейму
products["image_url"] = products.apply(copy_and_rename_image, axis=1)

upload\iblock\965\h6v1v60gi1ryckdh0tc7ay6ltnzwoi4a\2021-30-03_20-50-08__xDs6gko.webp
upload\iblock\da3\klvvib5jemjl1rp5rfraxiri73alqgtu\Снимок.PNG
upload\iblock\3bc\4b6qcru899yl23shyk2ndyfvic7eyqbp\s-l1600.jpg
upload\iblock\f18\n4qjnqyb01i5awupzy87xbd8w15z8dv5\Jobb-oldali-toloajto-zarszerkezet-ford-Transit-2000-2014-1766551-1-500x500.jpg
upload\iblock\30a\qpkbax69dk7oqivixfobnl5w4c062bw3\Изображение WhatsApp 2022-08-15 в 09.46.43.jpg
upload\iblock\e86\ygemqttik565m282bywo8ulfn6b9z8it\загружено.jpg
upload\iblock\452\r7wzzft4n0fqbatcvn54kvftuu28l81c\s-l1200.jpg
upload\iblock\12f\uq636hl13yzuptvz73t2ifek9j3fgpib\367_133-923.webp
upload\iblock\f23\6l5jhx8atvnyc2ywadu4j1bpdujgm9jp.jpeg
upload\iblock\f0e\9w1s02hi6ub3vped4y4unx3vyhgqvgyu.jpeg
upload\iblock\531\3lfp0z0644fcq5ezbcx7ju35v3886lj7\Снимок.PNG
upload\iblock\580\7r7ccj0be646o4j3oygfaigcm3vzq9ao\zl1rd78AC4jm.jpg
upload\iblock\155\47a8m6xnncpc20jkcsxqxt1oyof4e1ex\загружено.jpeg
upload\iblock\764\wv9ohc8az2nlnbdqn4bu31cotodys7p6\загруже

In [49]:
products["image_url"] = (
    "https://storage.yandexcloud.net/tcf-images/products/" + products["image_url"]
)
products["image_url"] = products["image_url"].fillna(
    "https://storage.yandexcloud.net/tcf-images/default.svg"
)
products["is_deleted"] = False
products.drop(columns=["old_image"]).to_csv("out/products.csv", index=False)

## Предложения (20 000 -> 25 000 за 4 месяца)
Скрестить с прайс-листом для цен и кол-ва

In [33]:
offers = pd.read_csv("data/offers.csv")

offers = offers[
    [
        "IE_XML_ID",
        "IE_NAME",
        "IE_PREVIEW_PICTURE",
    ]
].rename(
    columns={
        "IE_NAME": "name",
        "IE_XML_ID": "offer_bitrix_id",
        "IE_PREVIEW_PICTURE": "old_image",
    }
)

offers = offers[
    (offers["name"] != "1")
    & (offers["name"] != "0")
    & (offers["name"] != "01")
    & (offers["name"].notna())
]

In [34]:
offers

,offer_bitrix_id,name,old_image
0,F8-2,Focus II-III / C-Max / Kuga / Mondeo / S-Max 0...,/upload/iblock/d72/24sqmsxa5hpfyif8n4t4k5w1taf...
1,F8-3,Замок двери багажника Focus II-III,/upload/iblock/7b9/5vsxhfn3ve3rfwr3xm5fhcy63sy...
2,F8-4,Focus II-III / C-Max / Kuga / Mondeo / S-Max 0...,/upload/iblock/317/f1lpms9s3drhuewt5lyiwoyr1fd...
3,F8-5,Focus II-III / C-Max / Kuga / Mondeo / S-Max 0...,/upload/iblock/317/f1lpms9s3drhuewt5lyiwoyr1fd...
4,F8-6,Механизм замка Connect задней (высокой) двери ...,NaN
...,...,...,...
25024,378664,Шайба U-образная + Болт кардана (к-т),/upload/iblock/24b/05zn0stv265wbuygbzxzk13sujy...
25025,378665,Болт кардана M10x1x25mm,/upload/iblock/b1b/a6oxud5j5b7ujbakys3sfjbowdl...
25026,378666,1.25 / 1.4 / 1.6 Zetec-S 1326381,/upload/iblock/96e/hlpu2d4nm3lla3fzseljy0pmfov...
25027,378667,Прокладка ТНВД 2.0 TDCi 2003-,NaN


In [65]:
price_list = pd.read_excel("data/price_opt.xlsx")
price_list
# Проблемы в прайс-листе:
# - бренд - 5 пустых
# - Номер производителя - 19 пустых
# - цена опт - 27 пустых + 42, где цена = 1.0 руб
# - Того в прайсе 22,693 позиции 22,596 после чистки всех условий выше

,ID,Система,Подсистема,Название,Бренд,Номер производителя,Кроссы,Цена опт,Остаток
0,F2-123,Ford/Аксессуары,1,1,1,1,NaN,3609.0,0
1,F2-1,Ford/Аксессуары,Автомобильный набор,Автомобильный набор ARBORI Basic,AUTOFAMILY,AFEKBASIC01,AUTOFLEX NA0015,2408.0,0
2,F2-2,Ford/Аксессуары,Автомобильный набор,Автомобильный набор ARBORI Basic,AUTOFLEX,NA0015,AUTOFLEX NA0015,2284.0,0
3,F2-3,Ford/Аксессуары,Аптечка автомобильная,Аптечка,AIRLINE,AM04,NaN,253.0,0
4,F2-4,Ford/Аксессуары,Аптечка автомобильная,Аптечка,AIRLINE,AM01,NaN,630.0,0
...,...,...,...,...,...,...,...,...,...
22688,O1-185,Масла и автохимия/Тормозная жидкость,Тормозная жидкость,Жидкость тормозная Castrol BRAKE FLUID DOT 4 1Л,Castrol,15036B,"1776312, 1847947 - PFB401CN / 000000473402400;...",1033.0,3
22689,277112,Масла и автохимия/Тормозная жидкость,Тормозная жидкость,Жидкость тормозная DOT4 0.5 л. BOSCH,BOSCH,1987479106,000000473402400; 000000R403; 0009890807; 00098...,408.0,0
22690,278220,Масла и автохимия/Тормозная жидкость,Тормозная жидкость,Жидкость тормозная dot 4 - 0.5 VALEO 402402,VALEO\t\t,402402,000000473402400; 000000R403; 0009890807; 00098...,385.0,80
22691,276291,Масла и автохимия/Тормозная жидкость,Тормозная жидкость,ТОРМОЗНАЯ ЖИДКОСТЬ FORD 1847946 DOT 4,Ford,T207406,000000473402400; 000000R403; 0009890807; 00098...,619.0,0


In [66]:
price_list_clean = pd.read_excel("data/price_opt.xlsx")
price_list_clean = price_list_clean[
    [
        "ID",
        "Название",
        "Бренд",
        "Номер производителя",
        "Цена опт",
        "Остаток",
    ]
]
price_list_clean["ID"] = price_list_clean["ID"].astype(str)
price_list_clean = price_list_clean[
    (price_list_clean["Название"] != "1")
    & (price_list_clean["Название"] != 1)
    & (price_list_clean["Название"] != "0")
    & (price_list_clean["Название"] != 0)
    & (price_list_clean["Название"] != "01")
    & (price_list_clean["Цена опт"] != 1.0)
    & (price_list_clean["Цена опт"].notna())
    & (price_list_clean["Бренд"].notna())
    & (price_list_clean["Номер производителя"].notna())
    & (price_list_clean["Название"].notna())
]

In [73]:
dirty = price_list[
    ~price_list["ID"].astype(str).isin(price_list_clean["ID"].astype(str))
]

In [76]:
dirty["link"] = "https://ford-parts.com.ru/catalog/" + dirty["ID"].astype(str) + "/"

C:\Users\utikpuhlik\AppData\Local\Temp\ipykernel_20752\301094796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dirty['link'] = "https://ford-parts.com.ru/catalog/" + dirty["ID"].astype(str) + "/"


In [77]:
dirty

,ID,Система,Подсистема,Название,Бренд,Номер производителя,Кроссы,Цена опт,Остаток,link
0,F2-123,Ford/Аксессуары,1,1,1,1,NaN,3609.0,0,https://ford-parts.com.ru/catalog/F2-123/
627,F4-143,Ford/Двигатель,Вкладыши коленвала,1.6 D 1984-1988 коренные STD,KOLBENSCHMIDT,87274600,NaN,NaN,0,https://ford-parts.com.ru/catalog/F4-143/
764,F4-289,Ford/Двигатель,Вкладыши коленвала,2.3 D / 2.5 D Sierra / Scorpio шатунные STD,AE,AEB4615,NaN,NaN,0,https://ford-parts.com.ru/catalog/F4-289/
851,F4-390,Ford/Двигатель,Вкладыши коленвала,Transit 1986-2000 2.5 D / TD коренные +0.75,AE,AEM5437,844F 6K371 DB T117394,1.0,0,https://ford-parts.com.ru/catalog/F4-390/
1123,F4-1372,Ford/Двигатель,К-т болтов головки,К-т болтов головки 1.8 CVH,GOETZE,22-18006B,NaN,NaN,0,https://ford-parts.com.ru/catalog/F4-1372/
...,...,...,...,...,...,...,...,...,...,...
22574,O1-411,Масла и автохимия/Моторные масла,Масло двигателя,1,NaN,NaN,NaN,NaN,0,https://ford-parts.com.ru/catalog/O1-411/
22601,O1-78,Масла и автохимия/Моторные масла,Масло двигателя,5W30 4 / 5Л,1,1,5W/30 M2C913D F4LTR N / 5W/30 M2C913D F7L N / ...,1.0,0,https://ford-parts.com.ru/catalog/O1-78/
22602,O1-80,Масла и автохимия/Моторные масла,Масло двигателя,5W30 4 / 5Л,1,1,5W/30 M2C913D F4LTR N / 5W/30 M2C913D F7L N / ...,1.0,0,https://ford-parts.com.ru/catalog/O1-80/
22626,O1-69,Масла и автохимия/Моторные масла,Масло двигателя,5л. VAG VW LongLife 3 5W30,1,1,5W/30 M2C913D F4LTR N / 5W/30 M2C913D F7L N / ...,1.0,0,https://ford-parts.com.ru/catalog/O1-69/


In [63]:
x = price_list.merge(offers, left_on="ID", right_on="offer_bitrix_id", how="left")

In [54]:
x[x["Цена опт"] < 10]

,ID,Название,Бренд,Номер производителя,Цена опт,Остаток,offer_bitrix_id,name,old_image
5926,F8-656,Transit 1986-2000 Клипса планки задней распашн...,BSG,BSG30995024,9.0,684,F8-656,Transit 1986-2000 Клипса планки задней распашн...,/upload/iblock/296/066qv3nw5vmbuj3rga6ok82ksys...
6739,F8-1299,Клипса обшивки двери кабины Transit 2001->,LEX,WW0300,8.0,30,F8-1299,Клипса обшивки двери кабины Transit 2001->,NaN
6741,F8-1301,Клипса обшивки потолка TRANSIT 1991-2000,BSG,30-995-012,9.0,0,F8-1301,Клипса обшивки потолка TRANSIT 1991-2000,NaN
7817,F9-303,W5W (белая) / WY5W (желтая) 12V,NORD YADA,800045,8.0,0,F9-303,W5W (белая) / WY5W (желтая) 12V,/upload/iblock/64a/16bnuz71h0pohyrb2dilii4bdr6...


In [55]:
x

,ID,Название,Бренд,Номер производителя,Цена опт,Остаток,offer_bitrix_id,name,old_image
0,F2-1,Автомобильный набор ARBORI Basic,AUTOFAMILY,AFEKBASIC01,2408.0,0,F2-1,Автомобильный набор ARBORI Basic,NaN
1,F2-2,Автомобильный набор ARBORI Basic,AUTOFLEX,NA0015,2284.0,0,F2-2,Автомобильный набор ARBORI Basic,NaN
2,F2-3,Аптечка,AIRLINE,AM04,253.0,0,F2-3,Аптечка,NaN
3,F2-4,Аптечка,AIRLINE,AM01,630.0,0,F2-4,Аптечка,NaN
4,276085,Аптечка,AUTOFAMILY,TL8551,233.0,0,276085,Аптечка,/upload/iblock/090/i8cvqs13qnht9mp8k1kwfua0yjy...
...,...,...,...,...,...,...,...,...,...
22612,O1-185,Жидкость тормозная Castrol BRAKE FLUID DOT 4 1Л,Castrol,15036B,1033.0,3,O1-185,Жидкость тормозная Castrol BRAKE FLUID DOT 4 1Л,/upload/iblock/172/6mo7kuys1ltq8m2pa1yr3yaedz6...
22613,277112,Жидкость тормозная DOT4 0.5 л. BOSCH,BOSCH,1987479106,408.0,0,277112,Жидкость тормозная DOT4 0.5 л. BOSCH,/upload/iblock/182/mx77ugi0o6duep680xu9g71htah...
22614,278220,Жидкость тормозная dot 4 - 0.5 VALEO 402402,VALEO\t\t,402402,385.0,80,278220,Жидкость тормозная dot 4 - 0.5 VALEO 402402,/upload/iblock/5be/1z48id2ioobistqzqfopcrs8wvm...
22615,276291,ТОРМОЗНАЯ ЖИДКОСТЬ FORD 1847946 DOT 4,Ford,T207406,619.0,0,276291,ТОРМОЗНАЯ ЖИДКОСТЬ FORD 1847946 DOT 4,NaN
